In [28]:
import requests
from datetime import datetime, timedelta

In [29]:
interesting_classes = ['BUC: Lower Body Strength']
time_filter = None
day_filter = []
saved_classes = {}

In [41]:
def get_classes(studio_id = '952'):
    url = "https://api.bruce.app/v32/class"

    data = {"studio_id":studio_id,"start_time_after":datetime.strftime(datetime.now(),'%Y-%m-%dT00:00:00Z')}

    response = requests.request("GET", url, params=data)

    assert response.ok, print('Error in get_classes', response.status_code, response.text)

    return response.json()['classes']

In [56]:
classes = get_classes()


def process_classes(class_list,saved_classes):
    new_class = False
    for klass in class_list:
        try:
            assert klass['id'] not in saved_classes
            assert klass['title'] in interesting_classes
            assert klass['price'] is None
            assert klass['available_spots'] > 0
            assert klass['tier_level'] <= 2
            assert klass['deleted'] is False
            

            start_time = datetime.strptime(klass['start_time'], '%Y-%m-%dT%XZ') + timedelta(seconds=klass['time_offset'])
            created_at = datetime.strptime(klass['created_at'], '%Y-%m-%dT%XZ') + timedelta(seconds=klass['time_offset'])

            if time_filter is not None:
                assert start_time.hour >= time_filter
            if len(day_filter) > 0:
                assert start_time.day not in day_filter
            
            saved_classes[klass['id']] = {'created_at': datetime.strftime(created_at, '%Y-%m-%d %X'),
                                'title': klass['title'],
                                'start_time': datetime.strftime(start_time, '%A, %d %b at %H:%M'),
                                'saved': False}
            new_class = True
        except AssertionError:
            pass
    return saved_classes, new_class

In [57]:
import sqlite3

def create_db():
    # Create a database in RAM
    # db = sqlite3.connect(':memory:')

    # Creates or opens a file called mydb with a SQLite3 DB
    db = sqlite3.connect('db.sqlite3')

    ##########
    # CREATE #
    ##########
    cursor = db.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS classes(
            id INTEGER PRIMARY KEY,
            created_at TEXT,
            title TEXT,
            start_time TEXT
        )
    ''')
    db.commit()
    db.close()

create_db()
saved_classes, new_class = process_classes(classes, saved_classes)

In [58]:
def insert_db(saved_classes):
    db = sqlite3.connect('db.sqlite3')
    cursor = db.cursor()
    for class_id in saved_classes:
        if saved_classes[class_id]['saved'] is False:
            cursor.execute('''INSERT INTO classes(id, created_at, title, start_time)
                            VALUES(:id,:created_at, :title, :start_time)''',
                            {'id':class_id, 
                            'created_at':saved_classes[class_id]['created_at'], 
                            'title':saved_classes[class_id]['title'], 
                            'start_time':saved_classes[class_id]['start_time']
                            })
            saved_classes[class_id]['saved'] = True
    db.commit()
    db.close()
    return saved_classes
if new_class:
    saved_classes = insert_db(saved_classes)
    new_class = False

In [59]:
new_class

False

In [60]:
def get_db():
    db = sqlite3.connect('db.sqlite3')
    cursor = db.cursor()
    cursor.execute('''select * from classes''')

    result = cursor.fetchall()

    db.close()

    return result
get_db()

[(2396162,
  '2022-02-25 01:34:09',
  'BUC: Lower Body Strength',
  'Monday, 07 Mar at 06:30'),
 (2396164,
  '2022-02-25 01:34:09',
  'BUC: Lower Body Strength',
  'Monday, 07 Mar at 12:00'),
 (2413519,
  '2022-03-04 01:18:39',
  'BUC: Lower Body Strength',
  'Monday, 14 Mar at 06:30'),
 (2413521,
  '2022-03-04 01:18:39',
  'BUC: Lower Body Strength',
  'Monday, 14 Mar at 12:00')]